<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/40_3_%3D%3D_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Дообучи русскоязычную модель GPT для генерации фейковых новостей.
2. Датасет необходимо найти самостоятельно.
3. Приведи 5-10 примеров сгенерированных фейковых новостей на русском языке.
4. Подбери параметры генерации, улучшите качество новостей.
5. Изучи model.generate более детально.
6. Добейся такой генерации, чтобы не было обрыва мысли, новость заканчивалась либо точкой, либо восклицательным знаком, либо вопросительным (если уместно).



In [110]:
# Устанавливаем необходимые пакеты
%%capture
!pip install kaggle

In [111]:
# Импортируем необходимые библиотеки
import kagglehub
import pandas as pd
from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

In [112]:
# Определяем GPT модель и токенизатор
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(DEVICE)

In [113]:
path = kagglehub.dataset_download("vorvit/news-lenta-20-23")
print("Path to dataset files:", path)
df = pd.read_csv(path + "/Lenta_sample.csv", index_col=False)
df.head(1)

Path to dataset files: /root/.cache/kagglehub/datasets/vorvit/news-lenta-20-23/versions/1


,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,943560,https://lenta.ru/news/2020/01/01/trudovaya/,В России ввели электронные трудовые книжки,1577836860,1577866854,1,1,0,0,1,[2],https://icdn.lenta.ru/images/2019/12/30/15/201...,1577836860,Фото: Юрий Мартьянов /« Коммерсантъ» В России...,Работодатели смогут передавать сведения в Пенс...,Фото: Юрий Мартьянов /« Коммерсантъ» В ... кн...


In [114]:
df.columns

Index(['docid', 'url', 'title', 'modified', 'lastmodtime', 'type', 'domain',
       'status', 'part', 'bloc', 'tags', 'image_url', 'pubdate', 'text',
       'rightcol', 'snippet'],
      dtype='object')

In [115]:
df = df[["rightcol"]]

In [116]:
df = df[:10000]

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   rightcol  10000 non-null  object
dtypes: object(1)
memory usage: 78.3+ KB


In [118]:
df

,rightcol
0,Работодатели смогут передавать сведения в Пенс...
1,Соответствующая запись появилась в Instagram-а...
2,Вступил в силу принятый в 2018 году закон
3,Спящие слишком мало или слишком много более по...
4,Обнулить базу удастся при условии строго следо...
...,...
9995,Ан-2 при взлете не смог набрать высоту
9996,Вспышка заболевания приведет к сокращению рост...
9997,Каждый день он проводит у торгового центра
9998,Кадры распространило Главное управление МЧС по...


In [119]:
# Записываем строку в файл
with open('output.txt', 'w') as file:
    for index, row in df.iterrows():
        # Форматируем строку для записи
        line = f"{row['rightcol']}\n"
        file.write(line)

print("Строки DataFrame успешно записаны в отдельные текстовые файлы.")

Строки DataFrame успешно записаны в отдельные текстовые файлы.


In [120]:
# Загружаем и создаем датасет
train_path = '/content/output.txt'
train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=64)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [121]:
# Параметры для обучения
training_args = TrainingArguments(
    output_dir="./finetuned",       # директория с выходными данными
    overwrite_output_dir=True,      # перезаписываем содержимое выходной директории при каждом запуске
    num_train_epochs=50,            # число эпох обучения
    per_device_train_batch_size=32, # batch size для обучения
    per_device_eval_batch_size=32,  # batch size для выполнения
    warmup_steps=10,                # количество шагов для "прогрева" (управление скоростью обучения)
    gradient_accumulation_steps=16,) # накопление градиента (16 шагов накапливаем градиенты для batch_size, эмуляция вычисления на пакете 16 * 32 для слабых GPU)

# Инициализируем класс обучения
trainer = Trainer(
    model=model,                 # модель
    args=training_args,          # параметры обучения
    data_collator=data_collator, # загрузчик данных
    train_dataset=train_dataset, # датасет для обучения
    optimizers = (torch.optim.AdamW(model.parameters(), lr=1e-5), None)) # оптимизатор

In [122]:
# Запуск обучения
trainer.train()

Step,Training Loss


TrainOutput(global_step=0, training_loss=194518.2647705078, metrics={'train_runtime': 330.7806, 'train_samples_per_second': 32.197, 'train_steps_per_second': 0.151, 'total_flos': 928700694528000.0, 'train_loss': 194518.2647705078, 'epoch': 0})

In [123]:
text = "Не новость:"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
print()

In [147]:
do_sample=True           # Когда do_sample=True, модель использует стохастический выбор
num_beams=2              # Чем больше значение num_beams, тем выше вероятность найти лучшую последовательность
temperature=1.9          # Температура воздействует на вероятности. Если вам нужен более творческий и непредсказуемый текст, увеличьте температуру.
top_k=200                # Определяет максимальное количество наиболее вероятных токенов. Уменьшая количество рассматриваемых токенов, можно снизить вероятность появления бессмысленных или несвязанных предложений
top_p=0.9                # Все токены, чьи суммарные вероятности меньше этого порога, отбрасываются, и выбор следующего токена производится только из оставшихся.
min_length=50           # Минимальная длина текста
max_length=300           # Определяет максимальную длину генерируемого текста
no_repeat_ngram_size=3  # Запрет на повторение n-грамм

In [148]:
for i in range(1, 6):
    out = model.generate(input_ids,
                    do_sample=do_sample,
                    num_beams=num_beams,
                    temperature=temperature,
                    top_k=top_k,
                    top_p=top_p,
                    min_length=min_length,
                    max_length=max_length,
                    no_repeat_ngram_size=no_repeat_ngram_size,
                    eos_token_id=model.config.eos_token_id,)
    print(f"*****№{i}*****")
    generated_text = list(map(tokenizer.decode, out))[0]
    print(*generated_text.split("\n")[:1], '\n')

*****№1*****
Не новость: 

*****№2*****
Не новость: как мы в этом убедимся подробно расскажу, но сегодня я расскажу как работает такая система. 

*****№3*****
Не новость: я еще в июне заметил в своих контактах упоминание, что их любимый чай - Чайна глорифи. В какой-то день как всегда пошел дождь и хотелось выпить не в кафе, а в магазинчике с очень хорошим чаем (я по натуре своей не люблю большие супермаркеты). Под вечер, отоварившись, поехал домой.  

*****№4*****
Не новость: каждый пятый мужчина-психотерапевт в мире является отцом. (Brian Kerr, «Activity Changes Your Children to Talk», Daily Mail, June 25, 2007)  

*****№5*****
Не новость: мы не знаем истинных мотивов войны, но можем лишь предполагать, когда и почему это случилось. Если бы кто‑нибудь мог просто, ясно и четко ответить, каким именно образом случилось то, что случилось... 

